In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import pipeline
import torch
protgpt2 = pipeline('text-generation', model="nferruz/ProtGPT2", device = torch.device('cuda'))

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/655k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [ ]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/protgpt2_unconditional/generation_metadata_protgpt2.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")


len_dist_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/protgpt2_unconditional/uniref50_length_dist_protgpt2.json"

if os.path.exists(len_dist_filepath):
  with open(len_dist_filepath, "r") as f:
    uniprot_length_dist =  json.load(f)
  print("Loaded length distribution from drive")
else:

  #https://www.uniprot.org/uniprotkb/statistics#sequence-size
  bins = np.array([13,51,101,151,201,251,301,351,401,451,501,551,601,651,701,751,801,851,901,951,1001,1101,1201,1301,1401,1501,1601,1701,1801,1901,2001,2101,2201,2301,2401,2501,34350])
  swissprot_reviewed = np.array([0,9968,43534,59796,59574,58452,52413,52846,45901,37706,30572,22287,15830,13156,9403,7870,5700,4889,5301,4109,3007,4124,2897,2207,2070,1675,834,642,587,503,395,272,386,340,234,195,1462])
  TrEMBL_unreviewed = np.array([0,2668805,19825275,24705701,23838128,23462438,23225451,21389271,16814580,14287105,11501843,8283150,6266068,4715059,3755005,3186452,2687314,2166878,1843669,1457871,1153537,1975953,1398765,961048,664766,517536,390552,300984,236895,210921,180246,138808,122833,102865,82441,71548,527646])

  ecdf = np.cumsum(swissprot_reviewed) / np.sum(swissprot_reviewed)
  #shortest protein in uniprot is 14 res, longest is 34350 res.
  x = np.arange(14, 34350+1)
  ecdf = np.interp(x, bins, ecdf)

  # Sample from the empirical CDF
  num_samples = 11000
  random_values = np.random.rand(num_samples)
  sampled_lengths = np.round(np.interp(random_values, ecdf, x)).astype(int)
  #ten thousand sequences up to 1000 res in length
  sampled_lengths = sampled_lengths[sampled_lengths <= 1000][0:10000]

  # Plot the histogram of sampled values
  hist_values, bin_edges, patches = plt.hist(sampled_lengths, bins=x[0:1001-13], alpha=0.7, label='Sampled Values')
  plt.xlabel('X-axis label')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()

  uniprot_length_dist = list(zip([int(edge) for edge in bin_edges],[int(value) for value in hist_values]))
  with open(len_dist_filepath, "w") as f:
      json.dump(uniprot_length_dist, f)


Existing generation metadata read in.
Loaded length distribution from drive


In [ ]:
c = 0
i = 0
meta_data = {}
while i < 10000:
  torch.cuda.empty_cache()

  if all_metadata_df.empty:
    max_length = max([l[0] for l in uniprot_length_dist])
  else:
    sampling_lengths = {s: n for s, n in uniprot_length_dist if n > 0}
    for l in all_metadata_df["conditions"].str.extract(r'length = (\d+)', expand=False).astype(int):
      if l >= 14 and l in sampling_lengths.keys():
        sampling_lengths[l] = sampling_lengths[l] -1
    sampling_lengths = {s: n for s, n in sampling_lengths.items() if n > 0}
    #max_length = max(sampling_lengths.keys())
    #print("Max sampling length (Tokens, ~Res): " + str(max_length// 4) + ", " + str(max_length))

  #Annoyingly, the max_length parameter that protgpt2 takes is actually the token length which is variable (avg. 4 amino acids)
  #max_length = max_length // 3 #4

  max_length = 200

  meta_data['entity_id'] = str(uuid.uuid4())
  meta_data["batch_id"] = None
  meta_data["batch_size"] = None
  meta_data['output_file_name'] = None
  meta_data["Timestamp"] = str(datetime.now())
  meta_data['model'] = 'ProtGTP2'
  meta_data['task'] = 'sequence_generation'
  meta_data['wall_time_batch'] = None
  meta_data['gpu'] = 'T4 GPU'

  start_time = time.time()
  sequences = protgpt2("<|endoftext|>", max_length=max_length, do_sample=True, top_k=950, repetition_penalty=1.2, num_return_sequences=1, eos_token_id=0,)
  end_time = time.time()
  meta_data['wall_time_task'] = str(end_time-start_time) + " Seconds"
  #assuming only one sequence generated!!!
  sequence = sequences[0]['generated_text'].replace(" ", "").replace("<|endoftext|>", "").replace("\n", "")
  #print(sequence)
  meta_data['conditions'] = 'length = ' + str(length)
  meta_data['generated_sequence'] = sequence
  metadata_entry = pd.Series(meta_data)
  all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)
  i = i + 1
  if i % 5 == 0:
    all_metadata_df.to_csv(meta_data_filepath, index=False)
    print("Saved to metadata " + str(datetime.now()))


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Actual sampled length: 233


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Actual sampled length: 259


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Actual sampled length: 485


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Saved to metadata 2024-05-30 23:51:36.513445
Actual sampled length: 356


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
# prompt: kill colab runtime

!kill -9 -1
